# Recomendaci ́on Musical con Aprendizaje por
Refuerzo Basado en Preferencias del Usuario y
Estado Emocional

In [4]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import plotly.express as px
import warnings
import joblib
import random
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from transformers import BertTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import AdamW
from torch.nn import CrossEntropyLoss
warnings.filterwarnings(action = "ignore")

musicLyricsData = pd.read_csv('tcc_ceds_music.csv')


Analisis del dataset Music & Mental Health Survey Results

Pre-Procesamiento y entrenamiento de Detector de sentimientos

In [5]:
musicLyricsData.head()

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95.0,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.0
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51.0,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.0
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24.0,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.0
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54.0,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.0
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48.0,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.0


In [6]:
musicLyricsData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1214 entries, 0 to 1213
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                1214 non-null   int64  
 1   artist_name               1214 non-null   object 
 2   track_name                1214 non-null   object 
 3   release_date              1214 non-null   int64  
 4   genre                     1214 non-null   object 
 5   lyrics                    1214 non-null   object 
 6   len                       1213 non-null   float64
 7   dating                    1213 non-null   float64
 8   violence                  1213 non-null   float64
 9   world/life                1213 non-null   float64
 10  night/time                1213 non-null   float64
 11  shake the audience        1213 non-null   float64
 12  family/gospel             1213 non-null   float64
 13  romantic                  1213 non-null   float64
 14  communic

In [7]:
print(musicLyricsData.isnull().sum())

Unnamed: 0                  0
artist_name                 0
track_name                  0
release_date                0
genre                       0
lyrics                      0
len                         1
dating                      1
violence                    1
world/life                  1
night/time                  1
shake the audience          1
family/gospel               1
romantic                    1
communication               1
obscene                     1
music                       1
movement/places             1
light/visual perceptions    1
family/spiritual            1
like/girls                  1
sadness                     1
feelings                    1
danceability                1
loudness                    1
acousticness                1
instrumentalness            1
valence                     1
energy                      1
topic                       1
age                         1
dtype: int64


In [8]:
print('Number of duplicated rows: ' , len(musicLyricsData[musicLyricsData.duplicated()]))

Number of duplicated rows:  0


In [9]:
musicLyricsData.shape

(1214, 31)

In [10]:
unique = musicLyricsData.nunique().sort_values()
unique_values = musicLyricsData.apply(lambda x: x.unique())
pd.DataFrame({'Number of Unique Values': unique, 'Unique Values': unique_values})

,Number of Unique Values,Unique Values
Unnamed: 0,1214,"[0, 4, 6, 10, 12, 14, 15, 17, 20, 23, 28, 32, ..."
acousticness,662,"[0.9979919658553876, 0.9548192317462166, 0.840..."
age,19,"[1.0, 0.9857142857142858, 0.9714285714285714, ..."
artist_name,221,"[mukesh, frankie laine, johnnie ray, pérez pra..."
communication,1206,"[0.2637508813174463, 0.0012836971222831, 0.250..."
danceability,517,"[0.3577385465179248, 0.3317448283331528, 0.456..."
dating,1205,"[0.0005980861262889, 0.0355371338259024, 0.002..."
energy,612,"[0.1371101880258922, 0.2632402533492537, 0.139..."
family/gospel,1206,"[0.048857015216959, 0.0270074773775298, 0.0027..."
family/spiritual,1206,"[0.0005980861261369, 0.0511241990177646, 0.002..."


In [11]:
musicLyricsData = musicLyricsData[musicLyricsData.len > 7]

In [12]:
musicLyricsData["topic"].value_counts(normalize=True)

,proportion
topic,
sadness,0.298594
world/life,0.167907
romantic,0.149711
music,0.120761
violence,0.103391
obscene,0.079404
night/time,0.062035
feelings,0.018197


In [13]:
musicLyricsData = musicLyricsData[musicLyricsData['topic'] != 'feelings']

In [14]:
musicLyricsData["topic"].value_counts()

,count
topic,
sadness,361
world/life,203
romantic,181
music,146
violence,125
obscene,96
night/time,75


In [15]:
musicLyricsData["topic"].value_counts()

,count
topic,
sadness,361
world/life,203
romantic,181
music,146
violence,125
obscene,96
night/time,75


In [16]:
musicLyricsData = musicLyricsData[musicLyricsData['topic'] != 'feelings']

In [17]:
max_samples = 1517


balancedDataLyrics = (
    musicLyricsData
    .groupby('topic', group_keys=False)
    .apply(lambda x: x.sample(n=min(len(x), max_samples), random_state=42))
)

# Verificar el tamaño final de cada clase
print(balancedDataLyrics['topic'].value_counts())

topic
sadness       361
world/life    203
romantic      181
music         146
violence      125
obscene        96
night/time     75
Name: count, dtype: int64


In [18]:
balancedDataLyrics.to_csv("balanced_musicLyricsData.csv", index=False)

In [19]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples['lyrics'], padding='max_length', truncation=True)

# Aplicamos la tokenización
encoded_data = tokenizer(balancedDataLyrics['lyrics'].tolist(), padding=True, truncation=True, max_length=512)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [20]:
label_encoderLyrics = LabelEncoder()
balancedDataLyrics['topic_encoded'] = label_encoderLyrics.fit_transform(balancedDataLyrics['topic'])



In [21]:
class LyricsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Crear el Dataset
dfLyrics = LyricsDataset(encoded_data, balancedDataLyrics['topic_encoded'].tolist())

In [22]:
train_data, val_data = train_test_split(dfLyrics, test_size=0.2, random_state=42)


In [23]:
print(f'Máxima etiqueta: {balancedDataLyrics["topic_encoded"].max()}')

Máxima etiqueta: 6


In [24]:
numlabels = balancedDataLyrics["topic_encoded"].max() + 1
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=numlabels)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=16)



In [26]:
optimizer = AdamW(model.parameters(), lr=5e-5)

# Definir la función de pérdida
criterion = CrossEntropyLoss()

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 15  # Número de épocas de entrenamiento
patience = 3  # Número de épocas sin mejora antes de detener el entrenamiento
best_val_loss = float('inf')  # Inicializamos la mejor pérdida como infinita
no_improvement_counter = 0  # Contador de épocas sin mejora

model.train()  # Configura el modelo en modo entrenamiento

for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_dataloader:
        # Mover los datos al dispositivo (GPU/CPU)
        batch = {key: val.to(device) for key, val in batch.items()}

        # Hacer una predicción con el modelo
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        # Realizar retropropagación y optimización
        optimizer.zero_grad()  # Limpiar los gradientes anteriores
        loss.backward()  # Retropropagar los gradientes
        optimizer.step()  # Actualizar los parámetros

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader)}")

    # Evaluación después de cada época (opcional)
    model.eval()  # Configura el modelo en modo de evaluación
    eval_loss = 0
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():  # No se necesita calcular gradientes
        for batch in val_dataloader:
            batch = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            eval_loss += loss.item()
            predictions = torch.argmax(logits, dim=-1)
            correct_preds += (predictions == batch["labels"]).sum().item()
            total_preds += len(batch["labels"])

    val_loss = eval_loss / len(val_dataloader)
    print(f"Validation Loss: {val_loss}")
    print(f"Validation Accuracy: {correct_preds / total_preds}")

    # Early Stopping Logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement_counter = 0  # Reiniciar el contador de paciencia
    else:
        no_improvement_counter += 1

    if no_improvement_counter >= patience:
        print(f"Early stopping activated at epoch {epoch + 1}")
        break  # Detener el entrenamiento si no hay mejora

    model.train()  # Volver al modo de entrenamiento


Epoch 1, Loss: 0.0005905598727016088
Validation Loss: 1.1142851567516725
Validation Accuracy: 0.819327731092437
Epoch 2, Loss: 0.0005619159855996259
Validation Loss: 1.121771320576469
Validation Accuracy: 0.819327731092437
Epoch 3, Loss: 0.0005033191589366955
Validation Loss: 1.128981858988603
Validation Accuracy: 0.819327731092437
Epoch 4, Loss: 0.0004761735865031369
Validation Loss: 1.1356577159216006
Validation Accuracy: 0.819327731092437
Early stopping activated at epoch 4


In [37]:
model.eval()  # Poner el modelo en modo de evaluación
val_loss = 0
correct_preds = 0
total_preds = 0
with torch.no_grad():  # No se necesita calcular gradientes
    for batch in val_dataloader:
        batch = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        val_loss += loss.item()
        predictions = torch.argmax(logits, dim=-1)
        correct_preds += (predictions == batch["labels"]).sum().item()
        total_preds += len(batch["labels"])

val_accuracy = correct_preds / total_preds
print(f"Validation Loss: {val_loss / len(val_dataloader)}")
print(f"Validation Accuracy: {val_accuracy}")


Validation Loss: 1.1356577159216006
Validation Accuracy: 0.819327731092437


In [39]:
# Guardar el modelo y el tokenizador en el directorio local de Colab
model.save_pretrained('/content/proyectoFinal')
tokenizer.save_pretrained('/content/proyectoFinal')


('/content/proyectoFinal/tokenizer_config.json',
 '/content/proyectoFinal/special_tokens_map.json',
 '/content/proyectoFinal/vocab.txt',
 '/content/proyectoFinal/added_tokens.json')